In [ ]:
import os
import glob
import shutil
import subprocess
import qiita_files
from qiita_files.demux import to_per_sample_files

# set variables
base_dir = os.path.expanduser("~/Praktikum_Janssen")
demux = os.path.join(base_dir, "TestDemux", "seqs.demux")
out_dir = os.path.join(base_dir, "TestOutDir")
fastqc_outdir = os.path.join(base_dir, "Fastqc")
multiqc_outdir = os.path.join(base_dir, "Multiqc")

# conda environments
fastqc_env="fastqc"
multiqc_env="multiqc"

# TODO: check fastq format
#       if not fastq exit plugin
#       fastqc runs only on fastq files

# split demux into fastx-files
n_jobs=1
out_format="fastq"
to_per_sample_files(demux, samples=None, out_dir=out_dir, n_jobs=n_jobs, out_format=out_format)

# run fastqc/multiqc

# path to input fastqc_files
fastqc_indir=out_dir
suffixes = ("*.fastq", "*.fq", "*.fastq.gz", "*.fq.gz")
fastqc_files=[f for pattern in suffixes for f in glob.glob(f"{fastqc_indir}/{pattern}")]
# testcase print(fastqc_files)

# 1. create output dir
os.makedirs(fastqc_outdir, exist_ok=True)

# 2. run fastqc in conda env
for file in fastqc_files:
    subprocess.run(["conda", "run", "-n", fastqc_env, "fastqc", file, "--outdir", fastqc_outdir], check=True)
print(f"✅ FastQC abgeschlossen. Ausgabedateien befinden sich in: {fastqc_outdir}")

# output folder multiqc
multiqc_outdir=os.path.expanduser("~/Praktikum_Janssen/Multiqc")
multiqc_indir=fastqc_outdir

# 3. run multiqc in conda env
subprocess.run(["conda", "run", "-n", multiqc_env, "multiqc", "-o",multiqc_outdir, multiqc_indir], check=True)
print(f"✅ MultiQC abgeschlossen. Ausgabedateien befinden sich in: {multiqc_outdir}")

# 4. Aufräumen – FASTQ-Dateien löschen
for f in fastqc_files:
    try:
        os.remove(f)
    except OSError as e:
        print(f"⚠️ Fehler beim Löschen von {f}: {e}")

# 5. Aufräumen – FastQC-Reports löschen
for report in glob.glob(f"{fastqc_outdir}/*_fastqc.*"):
    try:
        os.remove(report)
    except OSError as e:
        print(f"⚠️ Fehler beim Löschen von {report}: {e}")

# (Optional) FastQC-Ausgabeordner selbst löschen, falls du nichts behalten willst:
try:
    os.rmdir(fastqc_outdir)
except OSError:
    pass  # wird ignoriert, falls Ordner nicht leer ist oder bereits gelöscht wurde

print("🧹 Aufräumen abgeschlossen – nur MultiQC-Ergebnisse wurden behalten.")




✅ FastQC abgeschlossen. Ausgabedateien befinden sich in: /home/tobias/Praktikum_Janssen/Fastqc
✅ MultiQC abgeschlossen. Ausgabedateien befinden sich in: /home/tobias/Praktikum_Janssen/Multiqc
🧹 Aufräumen abgeschlossen – nur MultiQC-Ergebnisse wurden behalten.


In [14]:
import h5py

def get_file_format(demux_path):
    """Gibt das Format der HDF5-Datei zurück (fastq oder fasta).

    Parameter
    ----------
    demux_path : str
        Pfad zur HDF5-Demux-Datei (.demux)

    Rückgabewert
    -------
    str
        "fastq" für FASTQ-Daten, "fasta" für FASTA-Daten
    """
    with h5py.File(demux_path, 'r') as f:
        # Überprüfen, ob das Attribut 'has-qual' existiert und True ist
        if f.attrs.get('has-qual', False):
            return "fastq"
        else:
            return "fasta"

file_format = get_file_format(demux)
print(f"Die Datei enthält {file_format.upper()}-Daten.")


Die Datei enthält FASTQ-Daten.
